### Use this to host an ollama server in Kaggle
Use the ngrok tunnel url as the base URL for your ollama Client

In [1]:
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

!curl -fsSL https://ollama.com/install.sh | sh

%pip install pyngrok ollama

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]          
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]0m 
Get:9 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [7,219 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]m
Get:12 ht

In [3]:
import time
import subprocess
import warnings
import ollama
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient

NGROK_AUTH_TOKEN = UserSecretsClient().get_secret("ngrok")
NGROK_USER = '<<HTTPBasicAuth Username>>'
NGROK_PASS = '<<HTTPBasicAuth Password>>'
NGROK_URL = '<<Enter ngrok tunnel URL>>'


def serve_ollama() -> None:
    """ Serves ollama if not already running """
    def is_ollama_up() -> bool:
        pname = 'ollama'
        try:
            call = subprocess.check_output(f"pidof {pname}", shell=True)
            return True
        except subprocess.CalledProcessError:
            return False
        
    if is_ollama_up():
        warnings.warn("Ollama already running. Proceeding without re-deployment...")
    else:
        process = subprocess.Popen(
            "nohup ollama serve",
            shell=True,
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE
        )
        time.sleep(5)
        print(f"Ollama deployed (PID: {process.pid}).")


def pull_ollama_model(model: str) -> None:
    """ Use if you need to manually pull a model locally. """
    subprocess.check_call(f"ollama pull {model}", shell=True)

def serve_ngrok():
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    ngrok_tunnel = ngrok.connect(
        addr="11434",
        proto='http',
        auth=f"{NGROK_USER}:{NGROK_PASS}",
        host_header="localhost:11434",
        url=NGROK_URL,
    )
    print(f"{ngrok_tunnel=}")

In [ ]:
opid = serve_ollama()
tunnel = serve_ngrok()

In [5]:
# ngrok.disconnect('https://bd07-35-204-215-101.ngrok-free.app')